### Поиск LOV и его соседей

In [168]:
from bs4 import BeautifulSoup
'''Доступ к Uniprot'''
from Bio import ExPASy
from Bio import SwissProt
import pandas as pd


In [59]:
with open("With_Cys/Cys_sequences_2.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')

sequences = soup.sequences.findAll('sequence')

In [60]:
seqs = {}
for seq in sequences:
    for seg in seq.find_all('seg'):
        if bool(seg.find(idx = '0')) == True and bool(seg.find(idx = '1')) == True:
            hits = seg.find_all('hit')
            if float(hits[0]['pvalue']) <= 1e-15 and float(hits[1]['pvalue']) <= 1e-15:
                seqs.setdefault(seq['name'], []).append([int(hits[0]['pos']), int(hits[1]['pos'])])

In [61]:
with open('With_Cys/final_proteins_2.fasta', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line.lstrip('>')
        else:
            fasta.setdefault(name, []).append(line)

In [45]:
new_seq = {}
for key in seqs.keys():
    if key in fasta:
        new_seq[key] = seqs[key]
seqs = new_seq

In [72]:
data = pd.read_csv('With_Cys/domains_2.csv')

In [74]:
data = data[data['seq id'].isin(seqs.keys())].reset_index(drop = True)

In [75]:
data.drop('Unnamed: 0', axis = 1, inplace = True)
data.head()

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan,kingdom
0,A0A1H3L262,59.0,151.0,49.0,155.0,PF13426.7,PAS_9,Domain,8.0,100.0,104.0,64.8,7.100000e-18,1.0,CL0183,Bacteria Proteobacteria
1,A0A1Q3I4W4,16.0,121.0,15.0,122.0,PF13426.7,PAS_9,Domain,2.0,103.0,104.0,79.0,2.600000e-22,1.0,CL0183,Bacteria Proteobacteria
2,A0A1F1PYR5,16.0,120.0,15.0,121.0,PF13426.7,PAS_9,Domain,2.0,103.0,104.0,72.3,3.300000e-20,1.0,CL0183,Bacteria Proteobacteria
3,A0A1G3K5R8,16.0,107.0,6.0,111.0,PF13426.7,PAS_9,Domain,8.0,99.0,104.0,65.9,3.200000e-18,1.0,CL0183,Bacteria Proteobacteria
4,A0A1G3K5R8,130.0,185.0,129.0,185.0,PF00196.19,GerE,Domain,2.0,57.0,57.0,58.6,3.300000e-16,1.0,CL0123,Bacteria Proteobacteria


In [76]:
data.shape

(4510, 16)

#### Находим LOV 

In [77]:
def IfLOV(row):
    b = False
    if row['clan'] == 'CL0183':
        for hit in seqs[row['seq id']]:
            if (int(row[3]) - hit[0]) <=5 and (hit[1] + 43 - int(row[4])) <= 1:
                return 'LOV'
    return row['hmm name']
        

In [78]:
data['hmm name'] = data.apply(IfLOV, axis = 1)

#### Находим Tandem LOV

In [79]:
from tqdm import tqdm

for name in tqdm(seqs.keys()):
    indexes = data[(data['seq id'] == name) & (data['hmm name'] == 'LOV')].index.values
    for i in indexes[:-1]:
        if (data.loc[i+1, 'hmm name'] == 'LOV' and 
            int(data.loc[i+1, 'envelope start']) - int(data.loc[i, 'envelope end']) <= 125):
            data.loc[i, 'envelope end'] = data.loc[i+1, 'envelope end']
            data.loc[i, 'hmm name'], data.loc[i+1, 'hmm name'] = 'TLOV', '_LOV'

100%|██████████| 1477/1477 [00:02<00:00, 506.25it/s]


In [82]:
data = data.loc[data['hmm name'] != '_LOV']

Отсеяла все лишние ловы (те которые входят вторыми в TLOV) теперь необходимо записать получившееся в отдельный файл, и уже теперь разбивать по царствам

In [83]:
tfile = open('With_Cys/domains_LOV_2.csv', 'a')
tfile.write(data.to_csv())
tfile.close()

In [ ]:
for name in tqdm(seqs.keys()):
    indexes = data[(data['seq id'] == name) & (data['hmm name'] == 'LOV')].index.values
    for i in indexes[:-1]:
        if (data.loc[i+1, 'hmm name'] == 'LOV' and 
            int(data.loc[i+1, 'envelope start']) - int(data.loc[i, 'envelope end']) <= 125):
            data.loc[i, 'envelope end'] = data.loc[i+1, 'envelope end']
            data.loc[i, 'hmm name'], data.loc[i+1, 'hmm name'] = 'TLOV', '_LOV'

In [81]:
data[data['hmm name'] == 'TLOV'].shape

(18, 16)